In [26]:
import numpy as np
from numpy import nanmedian
from scipy.fft import fft
import math

In [3]:
#' filt5
#' @description  Finds maximum of a 5x5 sliding window. If the central pixel is the maximum, this is flagged as a one. All other pixels are flagged as zero. Usually this function is not used directly.
#' @usage filt5(lon, lat, ingrid, nodata = NA)
#' @param lon longitude (vector) of the satellite image
#' @param lat latitude (vector) of the satellite image
#' @param ingrid The satellite data (matrix)
#' @param nodata value representing 'no data'
#'
#' @return returns a grid of zeros and ones
#' @seealso \code{\link{boa}}
#' @export
#'
#' @examples 
#' # none
filt5 <-
function(lon , lat, ingrid, nodata = NA){
#======================================================#
# Find peaks in 5 directions. FLag as 5
#======================================================#
 #outgrid = matrix(0,nrow=dim(ingrid)[1],ncol=dim(ingrid)[2] ) # make all zeros..
 nodatidx = ingrid[ingrid==nodata]    # creates single dim array with as much values as the matrix ingrid, with NANs
 ingrid[nodatidx] = -9999
 outgrid = ingrid*0 ## outgrid is a matrix with the shape of ingrid, full of Zeros
 l1 = length(lon)
 l2 = length(lat)
for(i in 3:(l1-2)){
 for(j in 3:(l2-2)){
   subg = ingrid[(i-2):(i+2),(j-2):(j+2)]   #results in slice with data from row i-2 to i+2 (i-2,i-1, i, i+1, i+2) (included) and columns (j-2, j-1, j, j+1, j+2)
if(sum(is.na(subg))==25){
outgrid[i,j] = 0
}else{
vec = as.vector(subg)    ## convert matrix to vector
ma = which.max(subg)
mi = which.min(subg)

   if(ma==13||mi==13){
 outgrid[i,j] = 1
}else{
  outgrid[i,j] = 0
}
}
   }
  }
 ## Fodass não tou a perceber para que serve esta parte  
outgrid[nodatidx] = nodata  
outgrid

SyntaxError: invalid syntax (2922168994.py, line 15)

In [4]:
def filt5(lon, lat, ingrid, nodata=np.nan):
    """
    Find peaks in 5 directions. FLag as 5
    """
    
    nodatidx = ingrid.flatten()*np.nan ## creates single dim array with as much values as the matrix ingrid, with NANs
    outgrid = np.zeros(ingrid.shape)   ## outgrid is a matrix with the shape of ingrid, full of Zeros
    
    l1 = len(lon)
    l2 = len(lat)
    
    for i in range(3, l1-2):   #tem de ser -1 em vez de -2 
        for j in range(3, l2-2):
            subg = ingrid[(i-2):(i+3), (j-2):(j+3)] # slice with data from row (i-2,i-1, i, i+1, i+2) and columns (j-2, j-1, j, j+1, j+2)
            if np.isnan(subg).sum()==25:    #if all values in submatrix subg are null values:
                outgrid[i,j] = 0
            else:
                vec = np.array(subg).flatten()    ## convert matrix to vector
                ma = np.where(subg == np.amax(subg))
                mi = np.where(subg == np.amin(subg))
                
                if ma==(2,2) or mi==(2,2):     #se se tratar do valor meio da matrix
                    outgrid[i,j] = 1
                else:
                    outgrid[i,j] = 0
    
    #Não estou a perceber esta parte
    #outgrid[nodatidx] = nodata
    return outgrid
                    
                
                
    
    

In [ ]:
#' filt3
#'
#' @param lon longitude (vector) of the satellite image
#' @param lat latitude (vector) of the satellite image
#' @param ingrid The satellite data (matrix)
#' @param grid5 results of \code{\link{filt5}}
#'
#' @return returns a median smoothed grid of satellite data
#' @seealso \code{\link{boa}}
#' @usage filt3(lon, lat, ingrid, grid5 = grid5(lon, lat, ingrid))
#' @export
#'
#' @examples
#' # none here
filt3 <-
function(lon , lat, ingrid, grid5){
#======================================================#
# Find peaks in 3 directions. FLag as 3
#======================================================#
 outgrid = ingrid*0 # make all zeros..
 l1 = length(lon)
 l2 = length(lat)
for(i in 3:(l1-2)){
 for(j in 3:(l2-2)){
  if((grid5[i,j]==0)==T){
 subg = ingrid[(i-1):(i+1),(j-1):(j+1)]
if(sum(is.na(subg))==9){
outgrid[i,j] = ingrid[i,j]
}else{
vec = as.vector(subg)
ma = which.max(subg)
mi = which.min(subg)

   if((ma==5||mi==5)==T){
 outgrid[i,j] = median(subg,na.rm=T)  # apply median filter if there is peak 3
}else{
  outgrid[i,j] = ingrid[i,j]
}
}
}else{
 outgrid[i,j] = ingrid[i,j]
}

 }
}
outgrid
}


In [ ]:
def filt3(lon, lat, ingrid, grid5):
    """
    Find peaks in 3 directions. FLag as 3
    """
    
    outgrid = ingrid*0   #make all zeros. in python if matrix ingrid has null values, they'll stay null after (*0)
    l1=len(lon)
    l2=len(lat)
    
    for i in range(3, l1-2):   #tem de ser -1 em vez de -2 
        for j in range(3, l2-2):
            if (grid5[i,j]==0)==True:
                subg = ingrid[(i-1):(i+2), (j-1):(j+2)]
                if np.isnan(subg).sum()==9:
                    outgrid[i,j] = ingrid[i,j]
                else:
                    vec = np.array(subg).flatten()    ## convert matrix to vector
                    ma = np.where(subg == np.amax(subg))
                    mi = np.where(subg == np.amin(subg))
                    
                    if ((ma==(1,1) or mi==(1,1))):     #pixel do meio
                        outgrid[i,j] = nanmedian(subg)
                    else:
                        outgrid[i,j] = ingrid[i,j]
            
            else:
                outgrid[i,j] = ingrid[i,j]
                
    return outgrid

In [ ]:
#' boa
#'
#' @param lon longitude (vector) of the satellite image
#' @param lat latitude (vector) of the satellite image
#' @param ingrid The satellite data (matrix). If using chlorophyll, transform using log(ingrid))
#' @param nodata value representing 'no data'
#' @param direction Logical. Should direction be calculated and returned
#'
#' @return either a stand alone asc grid of front gradient data or a list of:
#'		grdir : ascii grid Gradient direction
#'    front : Gradient magnitude. In the case of chlorophyll, this is a ratio
#' @references Belkin, I. M. & O'Reilly, J. E. An algorithm for oceanic front detection in chlorophyll and SST satellite imagery. Journal of Marine Systems, 2009, 78, 319 - 326
#' @details Note: These grids are in raster format, as is used in \code{\link{raster}}. 
#' @keywords sst, chl, front, satellite
#' @export
#'
#' @examples # none
boa <-
function(lon, lat, ingrid, nodata = NA, direction = F){

## Workhorse filter from EBImage. Modified so we don't need colorspace and other annoying requirements.		
	.filter2 <-
		function (x, filter) 
		{
			# Workhorse filter from EBImage. Modified so we don't need colorspace and other annoying requirements.
			validObject(x)
			validObject(filter)
			#if (colorMode(x) == TrueColor) 
			#   stop("this method doesn't support the 'TrueColor' color mode")
			dx = dim(x)
			#cmx = colorMode(x)
			df = dim(filter)
			if (any(df%%2 == 0)) 
				stop("dimensions of 'filter' matrix must be odd")
			if (any(dx[1:2] < df)) 
				stop("dimensions of 'x' must be bigger than 'filter'")
			cx = dx%/%2
			cf = df%/%2
			wf = matrix(0, nr = dx[1], nc = dx[2])
			wf[(cx[1] - cf[1]):(cx[1] + cf[1]), (cx[2] - cf[2]):(cx[2] + cf[2])] = filter
			wf = fft(wf)
			dim(x) = c(dx[1:2], prod(dx)/prod(dx[1:2]))
			index1 = c(cx[1]:dx[1], 1:(cx[1] - 1))
			index2 = c(cx[2]:dx[2], 1:(cx[2] - 1))
			pdx = prod(dim(x)[1:2])
            
            ## Não estou a conseguir implementar esta parte
			y = apply(x, 3, function(xx) {
				dim(xx) = dx[1:2]
				Re(fft(fft(xx) * wf, inverse = T)/pdx)[index1, index2]
			})
			dim(y) = dx
			y
		}	
	
	# browser()
    
    
#======================================================#
# Main BOA algorithm
#======================================================#
# require(adehabitat)
# x component (longitudinal kernel)
gx = matrix(c(-1,0,1,-2,0,2,-1,0,1),nrow=3, byrow=T)
# y component (latitudinal kernel)
gy = matrix(c(1,2,1,0,0,0,-1,-2,-1),nrow=3, byrow=T)
# filt 5 and 35 don't like NA's... but final steps are ok with it!

# if nodata is numeric, this will take care of it..
ingrid[ingrid == nodata] = -9999      ##NÂO ESTOU A PERCEBER   
# if something else: 
if(any(is.infinite(ingrid)|is.nan(ingrid)|is.na(ingrid))){
ingrid[is.na(ingrid)] = -9999       ## TO FIND MISSING VALUES. IF THE VALUE IS NA RETURN TRUE OTHERWISE FALSE
ingrid[is.nan(ingrid)] = -9999      ##CHECK IF MATRIX HAS ANY NaN (NOT A NUMBER) VALUE AS ELEMENT
ingrid[is.infinite(ingrid)] = -9999    ## CHECK TO SEE IF THE VECTOR CONTAINS INFINITE VALUES AS ELEMENTS
}

# do the median filtering
grid5 = filt5(lon, lat, ingrid, nodata = nodata)
grid35 = filt3(lon, lat, ingrid, grid5)

# make an index of bad values and land pixels.
grid35[grid35==-9999] = NA
naidx = is.na(grid35)
# convert these to zeros for smoothing purposes
grid35[naidx]=0
# perform the smoothing (Sobel filter)
tgx = .filter2(grid35, gx)
tgy = .filter2(grid35, gy)

# NOTE!! IDL CONVOL uses NORMALIZE function which defaults to the abs(sum(KERNEL)), in this case gx and gy
tx = tgx/sum(abs(as.vector(gx)), na.rm=T)
ty = tgy/sum(abs(as.vector(gy)), na.rm=T)
front = (sqrt(tx^2+ty^2))

    
#======================================================#
# landmask and edge dilation
#======================================================#
land = naidx*1
# land = naidx*1
land[land==1] = NaN
land[!is.nan(land)] = 1
# # use adehabitat library for image erosion (expand land pixels slightly)
# mask = morphology(as.asc(land, min(lon), yll = min(lat), cellsize = diff(lat)[1]), 'erode', 1)
# # remove edge pixels
# l1 = length(lon)
# l2 = length(lat)
# midx = mask*NaN
# midx[5:(l1-3), 5:(l2-3)]=1
# mask = mask*midx
# # account for the edge effect
# front = front *mask

    
    
#======================================================#
# landmask and edge dilation using raster!
#======================================================#


l1 = length(lon)
l2 = length(lat)

midx = land*NaN

midx[5:(l1-3), 5:(l2-3)] = 1

# land = land*midx

land = (land*midx)

# ssr = flip(t(raster(front)), 2)
ssr = flip(raster((t(front))),2)
extent(ssr) = c(min(lon), max(lon), min(lat), max(lat))

mask = flip(focal(raster(t(land)), w = matrix(c(0,0,0,0,1,0,0,0,0), nrow=3, ncol=3)), direction = 2)
extent(mask) = c(min(lon), max(lon), min(lat), max(lat))

front = mask*ssr


if(direction==T){
# ;   ************************************
# ;   *** Calculate Gradient Direction ***
# ;   ************************************
    GRAD_DIR = atan2(tgy@.Data, tgx@.Data)

# ;===> change radians to degrees
GRAD_DIR = GRAD_DIR*180/pi

# ;===> Adjust to 0-360 scheme (make negative degrees positive)
OK = which(GRAD_DIR < 0)
if(length(OK)>1)GRAD_DIR[OK] = 360 - abs(GRAD_DIR[OK])

# ;===> Convert degrees so that 0 degrees is North and East is 90 degrees
GRAD_DIR = (360 - GRAD_DIR + 90) %% 360

GRAD_DIR = flip(raster(t(GRAD_DIR)), direction = 2)
extent(GRAD_DIR) = extent(front)

list(grdir = GRAD_DIR*mask, front = front)
}
else{
front
}
}

In [76]:
import math

dx = np.matrix([[True,False,True,False],[False,False,False,False]])
print(dx)
midx = dx*np.nan
midx[0:1, 0:1] = 1
midx
print(midx)


dx = np.multiply(dx, midx)    #dx(2,4) midx(2,4)
dx

[[ True False  True False]
 [False False False False]]
[[ 1. nan nan nan]
 [nan nan nan nan]]


matrix([[ 1., nan, nan, nan],
        [nan, nan, nan, nan]])

In [ ]:
def boa(lon, lat, ingrid, nodata = np.nan, direction = False):
    
    def filter2(x, filter):
        
        dx = x.shape
        df = filter.shape
        
        if any(df%%2==0):
            exit("dimensions of 'filter' matrix must be odd")
        if (any(dx[1:3] < df)):
            exit("dimensions of 'x' must be bigger than 'filter'")
        
        cx = dx//2
        cf = df//2
        
        wf = np.zeros(shape=dx)
        wf[(cx[0]-cf[0]):(cx[0]+cf[0]), (cx[1]-cf[1]):(cx[1]+cf[1])] = filter
        wf = fft(wf)
        
        x.shape = [dx[0:2], math.prod(dx)/math.prod(dx[0:2])]
        index1 = [cx[0]:dx[0], 0:(cx[0] - 1)]
        index2 = [cx[1]:dx[1], 0:(cx[1] - 1)]
        
        pdx = math.prod(x.shape)
        

############################### Não estou a conseguir converter esta parte de R para python
			y = apply(x, 3, function(xx) {
				dim(xx) = dx[1:2]
				Re(fft(fft(xx) * wf, inverse = T)/pdx)[index1, index2]
			})
			dim(y) = dx
			y
		}	
##############################
        
    
#======================================================#
# Main BOA algorithm
#======================================================#      
    gx = np.matrix([[-1,0,1],[-2,0,2],[-1,0,1]])   
    gy = np.matrix([[1,2,1],[0,0,0],[-1,-2,-1]])
        
    np.nan_to_num(ingrid, nan=-9999, posinf=-9999, neginf=-9999) ##replace NaN and inf values with -9999
        
    grid5 = filt5(lon, lat, ingrid, nodata = nodata)
    grid35 = filt3(lon, lat, ingrid, grid5)

    # make an index of bad values and land pixels.
    grid35 = grid35.astype("float")
    grid35[grid35 == -9999]=np.nan
    naidx = np.isnan(grid35)       
    # convert these to zeros for smoothing purposes
    grid35[naidx]=0  

    # perform the smoothing (Sobel filter)
    tgx = filter2(grid35, gx)
    tgy = filter2(grid35, gy)
        
    tx = tgx/np.nansum(abs(np.array(gx).flatten()))
    ty = tgy/np.nansum(abs(np.array(gy).flatten()))
    front = (math.sqrt(tx**2+ty**2))


#======================================================#
# landmask and edge dilation
#======================================================#

    land = naidx*1
    land = land.astype("float")

    land[land==1] = np.nan
    land[~np.isnan(land)] = 1

    
#======================================================#
# landmask and edge dilation using raster!
#======================================================#

    l1=lon.size    #using size because lon and lat are matrices
    l2=lat.size

    midx = land*np.nan

    midx[5:(l1-2), 5:(l2-2)] = 1

    land = np.multiply(land, midx)

    ################################## Não consigo converter isto para python
    ssr = flip(raster((t(front))),2)
    extent(ssr) = c(min(lon), max(lon), min(lat), max(lat))

    mask = flip(focal(raster(t(land)), w = matrix(c(0,0,0,0,1,0,0,0,0), nrow=3, ncol=3)), direction = 2)
    extent(mask) = c(min(lon), max(lon), min(lat), max(lat))

    front = mask*ssr
    ##################################


    if direction==True:
#   ************************************
#   *** Calculate Gradient Direction ***
#   ************************************
        GRAD_DIR = atan2(tgy@.Data, tgx@.Data)     ## NÃO COMPREENDO
        # ata2() -> function returns the radian arctangent between the x-axis and the vector from the origin to (x, y).

        # ;===> change radians to degrees
        GRAD_DIR = GRAD_DIR*180/math.pi

        # ;===> Adjust to 0-360 scheme (make negative degrees positive)
        OK = np.where(GRAD_DIR < 0)     ## retorna index da matriz cujos valores são negativos

        if grad[OK].size >1:
            GRAD_DIR[OK] = 360 - abs(GRAD_DIR[OK])

        # ;===> Convert degrees so that 0 degrees is North and East is 90 degrees
        GRAD_DIR = (360 - GRAD_DIR + 90) % 360     

        GRAD_DIR = flip(raster(t(GRAD_DIR)), direction = 2)       ## NÃO CONSIGO CONVERTER PARA PYTHON
        extent(GRAD_DIR) = extent(front)    
        #extent -> recebe matriz com 4 elementos e retorna a class Extent com (xmin, xmax,ymin, ymax)

        list(grdir = np.multiply(GRAD_DIR, mask), front = front)   
    
    else:
        front
    



In [130]:
grad =np.matrix([[1,5,7],[-4,0,-1], [2,-3,4]])
mask =np.matrix([[1,1,1],[1,1,1], [1,1,1]])

grdir = np.multiply(grad, mask)
grdir

matrix([[ 1,  5,  7],
        [-4,  0, -1],
        [ 2, -3,  4]])

In [128]:
grad

matrix([[89, 85, 83],
        [94, 90, 91],
        [88, 93, 86]])

In [120]:
grad[OK].size

3

In [78]:
ssr = np.flipud(raster(25))

NameError: name 'raster' is not defined

In [55]:
naidx = np.matrix([[False, False, False],[False, True, False],[False, False, False]])

land = naidx*1
land = land.astype("float")
land[land==1] = np.nan
land[~np.isnan(land)] = 1
land
len(land)

3

In [172]:
gy = np.matrix([[1,2,1],[0,np.nan,0],[-1,np.inf,-1]])
gx = np.matrix([[1,2,1],[0,2,0],[-1,0,-1]])
px = np.matrix([[1,2,1],[0,2,0],[-1,1,-1]])

gy = np.nan_to_num(gy, nan=-9999, posinf=-9999, neginf=-9999) 
gy

matrix([[ 1.000e+00,  2.000e+00,  1.000e+00],
        [ 0.000e+00, -9.999e+03,  0.000e+00],
        [-1.000e+00, -9.999e+03, -1.000e+00]])

In [173]:
gy[gy==1] = np.nan
gy[~np.isnan(gy)]=1

In [180]:
gy[1:3,1:3]

matrix([[1., 1.],
        [1., 1.]])

In [170]:
y = gy*np.nan

In [171]:
y

matrix([[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]])

In [183]:
s = [[1,2,-1],[-9999,0,0],[-9999,-2,-1]]
s

[[1, 2, -1], [-9999, 0, 0], [-9999, -2, -1]]

In [185]:
s[5:9]

[]

In [154]:
gy[gy==-9999]=np.nan
np.isnan(gy)

matrix([[False, False, False],
        [False,  True, False],
        [False,  True, False]])

In [156]:
tx=2
ty=3
math.sqrt(tx**2+ty**2)

3.605551275463989

In [147]:
gz

[matrix([[1, 2, 1]], dtype=object),
 matrix([[0, None, 0]], dtype=object),
 matrix([[-1, -2, -1]], dtype=object)]

In [209]:
A = [[1,2,3],[4,5,6]]
A[0][1] = np.nan
A

[[1, nan, 3], [4, 5, 6]]

In [202]:
A[0]

nan

In [189]:
type(A)

list

In [211]:
a=np.array([[10,20,30],[40,50,60]])
a=a.astype('float')
a[a==10] = np.nan
a

array([[nan, 20., 30.],
       [40., 50., 60.]])